In [12]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# example of random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras


import tensorflow_addons as tfa
import tensorflow as tf

from scipy import stats
from imblearn.over_sampling import RandomOverSampler, SMOTE


In [13]:
# configs
EPOCHS = 5
BATCH_SIZE = 32

TIME_SERIES_SIZE = 4   # Determines the window size. Ex (4,9)
TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

baseFolder = "../data_2019_processed/"

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [21]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    return results

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        if(dataframe[column].dtypes in ()):
            dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    #for column in outputClasses:
    #    if(dataframe[column].dtypes != object):
    #        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset


def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [27]:
X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_undersample.csv")
#X_train  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_oversample.csv")

#AA = pd.read_csv(baseFolder+"allData-classification-numeric-normalized.csv")
#X_train, X_test = train_test_split(AA,test_size=0.25)

In [28]:
print(X_train.info())
X_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407451 entries, 0 to 407450
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            407451 non-null  float64
 1   location            407451 non-null  float64
 2   timestamp           407451 non-null  float64
 3   day_of_week         407451 non-null  float64
 4   light               407451 non-null  float64
 5   phone_lock          407451 non-null  float64
 6   proximity           407451 non-null  float64
 7   sound               407451 non-null  float64
 8   time_to_next_alarm  407451 non-null  float64
 9   minutes_day         407451 non-null  float64
 10  timestamp_text      407451 non-null  object 
 11  class               407451 non-null  object 
dtypes: float64(10), object(2)
memory usage: 37.3+ MB
None


,activity,location,timestamp,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day,timestamp_text,class
0,0.00,0.0,0.000869,0.0,0.000175,0.0,1.0,0.000000,0.982143,0.597637,2018-05-15 14:20:45,awake
1,0.00,0.0,0.000869,0.0,0.000175,0.0,1.0,0.000000,0.982143,0.597637,2018-05-15 14:20:45,awake
2,0.25,0.5,0.000869,0.0,0.000165,0.0,1.0,0.604408,0.982044,0.598332,2018-05-15 14:21:15,awake
3,0.25,0.5,0.000869,0.0,0.001449,0.0,1.0,0.604408,0.982044,0.598332,2018-05-15 14:21:45,awake
4,0.25,0.5,0.000869,0.0,0.000198,0.0,1.0,0.601849,0.981944,0.599027,2018-05-15 14:22:15,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
407446,0.25,1.0,0.006924,0.0,0.000000,1.0,1.0,0.644370,0.992956,0.549687,2018-06-12 13:11:39,awake
407447,0.25,1.0,0.006924,0.0,0.000000,1.0,1.0,0.644370,0.992956,0.550382,2018-06-12 13:12:09,awake
407448,0.25,1.0,0.006924,0.0,0.000538,1.0,1.0,0.624127,0.992857,0.551077,2018-06-12 13:13:37,awake
407449,0.00,1.0,0.006924,0.0,0.000000,0.0,0.0,0.540295,0.992758,0.551772,2018-06-12 13:14:07,awake


In [29]:
# transforms the input data to float32
X_test = transform_data_type(X_test)

# transforms the input data to float32
X_train = transform_data_type(X_train)


# balance
print(Counter(X_train['class']))
smote = SMOTE(random_state = 32)
X_train, y_train = smote.fit_resample(X_train[inputFeatures], X_train['class'])
X_train['class'] = y_train
print(Counter(X_train['class']))


# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the testing dataset
X_train = transform_output_nominal_class_into_one_hot_encoding(X_train)


Counter({'awake': 289267, 'asleep': 118184})
Counter({'awake': 289267, 'asleep': 289267})


In [30]:
# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

X_train_data, y_train_data = create_dataset_time_series_with_one_output(   #timestamp
    X_train_data, 
    y_train_data, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

X_test_data, y_test_data = create_dataset_time_series_with_one_output(    #timestamp
    X_test_data, 
    y_test_data, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)


print("shape: ",X_train_data.shape, y_train_data.shape)
print("Size: ",X_test_data.shape,y_test_data.shape)       

shape:  (578530, 4, 9) (578530, 2)
Size:  (136282, 4, 9) (136282, 2)


In [31]:
# transtorm data to tensor slices
test_dataset_series = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))
train_dataset_series = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))

2023-03-10 10:07:20.365963: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 10:07:20.427716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 10:07:20.427975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 10:07:20.431537: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [32]:
train_dataset_series


<TensorSliceDataset element_spec=(TensorSpec(shape=(4, 9), dtype=tf.float64, name=None), TensorSpec(shape=(2,), dtype=tf.uint8, name=None))>

In [33]:
#client_test_dataset.window(size=4, shift=1, stride=1, drop_remainder=True)
#test_dataset_series.batch(BATCH_SIZE) # usado no federated learning

In [34]:
# batch data size
#test_dataset_series1 = test_dataset_series.batch(1)
#train_dataset_series1 = train_dataset_series.batch(1)

test_dataset_series1 = test_dataset_series.batch(BATCH_SIZE)
train_dataset_series1 = train_dataset_series.batch(BATCH_SIZE)

train_dataset_series1

<BatchDataset element_spec=(TensorSpec(shape=(None, 4, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None, 2), dtype=tf.uint8, name=None))>

In [35]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 2, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/2


2023-03-10 10:07:26.124138: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2023-03-10 10:07:28.049568: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


18080/18080 [==============================] - 141s 8ms/step - loss: 0.4058 - categorical_accuracy: 0.8148
Epoch 2/2
4259/4259 [==============================] - 14s 3ms/step

awake
Accuracy: 0.728511
Precision: 0.764554
Recall: 0.885662
F1 score: 0.820664
Cohens kappa: 0.274384
ROC AUC: 0.697155
\Confusion Matrix
[[14627 26070]
 [10929 84656]]

asleep
Accuracy: 0.728988
Precision: 0.569103
Recall: 0.359413
F1 score: 0.440580
Cohens kappa: 0.273602
ROC AUC: 0.696426
\Confusion Matrix
[[84804 11012]
 [25922 14544]]

Global
2
accuracy:  0.7287499449670536
precision:  0.6668285450010212
recall:  0.6225374083303682
f1_score:  0.630622180838158
cohen_kappa_score:  0.27399277752510853
roc_auc_score:  0.6967903823955335


In [37]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 7, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/7
18080/18080 [==============================] - 142s 8ms/step - loss: 0.4048 - categorical_accuracy: 0.8153
Epoch 2/7
18080/18080 [==============================] - 150s 8ms/step - loss: 0.3536 - categorical_accuracy: 0.8399
Epoch 3/7
18080/18080 [==============================] - 147s 8ms/step - loss: 0.3290 - categorical_accuracy: 0.8508
Epoch 4/7
18080/18080 [==============================] - 147s 8ms/step - loss: 0.3090 - categorical_accuracy: 0.8596
Epoch 5/7
18080/18080 [==============================] - 145s 8ms/step - loss: 0.2934 - categorical_accuracy: 0.8668
Epoch 6/7
18080/18080 [==============================] - 148s 8ms/step - loss: 0.2823 - categorical_accuracy: 0.8724
Epoch 7/7
4259/4259 [==============================] - 10s 2ms/step

awake
Accuracy: 0.705537
Precision: 0.751993
Recall: 0.865659
F1 score: 0.804832
Cohens kappa: 0.217264
ROC AUC: 0.680609
\Confusion Matrix
[[13408 27289]
 [12841 82744]]

asleep
Accuracy: 0.70

In [38]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 15, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/15
 2777/18080 [===>..........................] - ETA: 2:00 - loss: 0.4951 - categorical_accuracy: 0.7582

KeyboardInterrupt: 

In [18]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 100, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
model.fit(train_dataset_series1, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/100
6896/6896 [==============================] - 53s 7ms/step - loss: 0.2948 - categorical_accuracy: 0.8796
Epoch 2/100
6896/6896 [==============================] - 50s 7ms/step - loss: 0.2989 - categorical_accuracy: 0.8755
Epoch 3/100
6896/6896 [==============================] - 49s 7ms/step - loss: 0.3014 - categorical_accuracy: 0.8727
Epoch 4/100
6896/6896 [==============================] - 59s 9ms/step - loss: 0.3000 - categorical_accuracy: 0.8781
Epoch 5/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2947 - categorical_accuracy: 0.8804
Epoch 6/100
6896/6896 [==============================] - 64s 9ms/step - loss: 0.2932 - categorical_accuracy: 0.8833
Epoch 7/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2925 - categorical_accuracy: 0.8837
Epoch 8/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2858 - categorical_accuracy: 0.8883
Epoch 9/100
6896/6896 [=============

In [20]:
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4259/4259 [==============================] - 11s 2ms/step

awake
Accuracy: 0.272685
Precision: 0.979019
Recall: 0.102880
F1 score: 0.186194
Cohens kappa: 0.038272
ROC AUC: 0.737497
\Confusion Matrix
[[25823   243]
 [98877 11339]]

asleep
Accuracy: 0.270953
Precision: 0.205180
Recall: 0.990630
F1 score: 0.339950
Cohens kappa: 0.037826
ROC AUC: 0.706877
\Confusion Matrix
[[11340 99114]
 [  242 25586]]

Global
2
accuracy:  0.2718187288123156
precision:  0.5920998003566356
recall:  0.5467550616729189
f1_score:  0.263071788036283
cohen_kappa_score:  0.03804903634117718
roc_auc_score:  0.7221871440907924
